## 기본 용어 정리
* Prompt: ChatGPT의 출력을 원하는 방향으로 유도하기 위한 입력 테스트
* Role: User, Assistant, System

## 간단한 원리
* Next Token Prediction
* ChatGPT같은 LLM모델은 기본적으로 정해진 수의 단어를 알고 있다.
    * 라마2 7B는 32000개, 젬마 7B는 256000개
* ChatGPT는 몇십만개 단어 중 1개를 예측하는 방식
* 단어가 우리가 생각하는 단어가 아니다. 정확히는 Token이다. ChatGPT는 한글이나 영어 문자를 보는 게 아니고 이걸 AI모델이 이해할 수 있는 숫자로 변환해야하는데 이 숫자들의 수입니다.

* 모델 : gpt-3.5-turbo-0124 (최신)
* temperature : 높을수록 동일한 Prompt에도 매번 다르게 이야기하는 경향이 있다. 0.0으로 세팅시 같은 답변으로만 대답

## OpenAI API 호출

In [1]:
from openai import OpenAI

OPENAI_API_KEY = "API키를 입력하세요"

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(model='gpt-3.5-turbo-0125',
                                            messages=[{'role': 'user', 'content': '왜 하늘은 하늘색인가요?'}],
                                            temperature=0.0)

print(completion.choices[0].message.content)

하늘은 하늘색인 이유는 대기 중의 분자들이 햇빛을 흡수하고 산란시키기 때문입니다. 태양으로부터 오는 빛은 다양한 파장을 가지고 있는데, 대기 중의 분자들은 이 빛을 흡수하고 다시 방출하면서 파장이 긴 파장인 파란색을 더 많이 산란시킵니다. 그래서 하늘은 파란색으로 보이게 되는 것입니다.


## 기본적인 Prompt 구조

프롬프트는 2가지 종류
1. 사용자가 ChatGPT한테 실제로 전달하는 User Prompt
2. User Prompt 이전에 오는 해당 LLM Aplication에 적합한 메타 System Prompt

System Prompt?
* User Prompt를 전달하기 전에 관련 맥란이나 지침을 설정하는 프롬프트
* 페르소나, 어조 등으로 설정
* 출력값 지정, 페르소나 및 어조 설정, 외부 정보 주입, 규칙 설정

왠만한 모델들은 Prompt 입력 시 기본 Systemp Prompt 있음

## System Prompt 호출

In [6]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[
        {'role': 'system', 'content': '당신은 물리선생님입니다. 저학년 학생들을 대상으로 친절하고 쉽게 설명해주세요.'},
        {'role': 'user', 'content': '왜 숲은 초록색인가요?'}],
        temperature=0.0
)

print(completion.choices[0].message.content)

숲이 초록색인 이유는 나무와 식물들이 엽록소라는 물질을 가지고 있기 때문이에요. 엽록소는 빛을 흡수하여 에너지를 만들어주는 역할을 해요. 특히, 엽록소는 녹색 빛을 흡수하고 다른 색의 빛을 반사하기 때문에 우리 눈에는 초록색으로 보여요. 그래서 숲이나 나무들이 초록색으로 보이는 거에요!


## User Prompt 호출

In [8]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[
        {'role': 'user', 'content': '당신은 물리선생님입니다. 저학년 학생들을 대상으로 친절하고 쉽게 설명해주세요. 왜 숲은 초록색인가요?'}
    ],
    temperature=0.0
)

print(completion.choices[0].message.content)

숲이 초록색인 이유는 주로 나무와 식물들이 엽록소라는 녹색 색소를 가지고 있기 때문입니다. 엽록소는 빛을 흡수하여 에너지를 생산하는 역할을 합니다. 빛 중에서 파란색과 녹색의 파장을 가장 많이 흡수하고, 녹색의 파장을 반사하기 때문에 우리 눈에는 숲이 초록색으로 보이는 것이죠. 따라서 숲은 주로 초록색으로 보이게 됩니다. 이렇게 숲이 초록색인 이유를 이해하면 숲의 생명력과 식물들의 중요성을 더욱 더 깊이 이해할 수 있을 거예요.


비슷한 답을 얻을 수 있다.
User Prompt는 유저가 직접 적어야 하지만, System Prompt는 System 개발자가 직접 맞춰서 작성한 Prompt이기 때문에 개발자에 맞춰져있다.

## Completion X, Stream O
* Completion의 뜻은 '완성'으로, 결과를 한번에 받음
* Stream은 실시간으로 바로바로 받을 수 있다는 특징이 있음
* ChatGPT가 문장을 완성하지 않고, 각 단어별로 바로바로 보여주는 방법

In [16]:
stream = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': '왜 숲은 초록색인가?'}],
    temperature=0.0,
    stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='')

숲이 초록색인 이유는 주로 나무와 식물들이 엽록소라는 녹색 색소를 가지고 있기 때문입니다. 엽록소는 빛을 흡수하여 에너지를 생산하는 역할을 하며, 이 과정에서 녹색을 반사하기 때문에 숲은 우리 눈에 초록색으로 보입니다. 또한, 숲은 자연적으로 식물이 많이 자라는 곳이기 때문에 초록색이 우세하게 나타나게 됩니다. 이러한 이유로 숲은 주로 초록색으로 보이게 됩니다.

* 실제 ChatGPT랑 대화하는 것처럼 나왔다

## 특징
1. Prompt는 영어로 해야 모델의 제성능을 발휘하는 편
   - ChatGPT, Claude 같은 모델들의 학습 데이터 중 큰 비중이 영어로 추정되기 때문
   - 학습 데이터가 공개된 라마1의 경우에도 대부분이 영어이며 한글은 극소량만 존재함
   - 한글 출력값이 필요하더라도 영어 Prompt를 통해 한글 출력값을 유도하는게 성능이 더 좋을 수 있음
2. AI 모델의 출력값은 입력값에 의존도가 매우 높음
   - 잘 한 것 같은데 원하는 결과가 안 나오면 입력이 모호하거나 필요한 내용이 빠졌을 수도 있음 (그게 아닌 경우 모델한테 태스크가 너무 어려울 수는 있음)
3. Prompt를 이렇게 저렇게 바꿨을 때 "더 좋아보이는" 결과보다는 특정 지표에서 유의미하게 더 좋거나 여러 번의 블라인드 테스팅을 통해 더 좋은 Prompt를 정하는 것을 추천
   - 다음 챕터인 프롬프트 엔지니어링 라이프사이클에서 자세하게 알려드릴 예정

## 정리
- Prompt, Role, Output
  - Role은 User, Assistant(ex. ChatGPT)
  - Prompt는 User, System Prompt
- ChatGPT 작동 원리 = Next Token Prediction
  - ChatGPT 같은 모든 LLM API는 단어1, 단어2, 단어3이 있을 때 단어3 뒤에 나올 가장 적합한 단어를 선택하는 식으로 출력
- OpenAI API 호출 시 model과 messages를 지정해줘야 하며 결과 재현을 위해서는 temperature=0.0 지정을 추천
  - 전체 결과값을 한꺼번에 받는 Completions, 실시간으로 바로바로 받을 수 있는 Stream으로 나뉨